# Local Linear Embedding

Author: Chris Nunn, Jeremy Rothschild

[Theory](#linkTheory)  <br>
[Example 1: The classic cinnamon bun](#linkExample1)  <br>
[Example 2: Faces](#linkExample2) <br>
 
[Resources](#linkResources)  

### Theory<a id='linkTheory'></a>

CONTINUE

#### LLE algorithm

1. Compute neighors of $X_i$
2. Compute the weights $W_{ij}$ that best reconstruct each data point $X_i$ from it's neighbors, minimzing the cost function $J=\sum_i |X_i + \sum_j W_{ij}X_j|^2$ with constraints.
3. Compute the (lower dimensional) vectors $Y_i$ best reconstructed by the weights $W_{ij}$, minimzing the cost function $J=\sum_i |Y_i + \sum_j W_{ij}Y_j|^2$, which is equivalent to solving an eigenvector problem.

#### Discussion


#### Code

In the following sections we have written some examples of LLE.

### Example 1: The cinnamon bun<a id='linkExample1'></a>

### Example 2: Faces<a id='linkExample2'></a>

### Ressources<a id='linkResources'></a>

[classic paper](https://cs.nyu.edu/~roweis/lle/papers/lleintro.pdf)